<a href="https://colab.research.google.com/github/satvik314/RAG_experiments/blob/main/HyDE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -qU langchain langchain-openai openai chromadb tiktoken faiss-cpu


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 798.0/798.0 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.1/212.1 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 509.0/509.0 kB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 32.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 38.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 34.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.6/216.6 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.3/48.3 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 23.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.3/60.3 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
!mkdir -p blog_posts
!unzip -q /content/langchain_blog_posts.zip -d blog_posts

In [2]:
import os
from getpass import getpass

os.environ["OPENAI_API_KEY"] = getpass()

··········


Hypothetical Document Embeddings (HyDE)

In [4]:
from langchain_openai import OpenAI, OpenAIEmbeddings
from langchain.chains import LLMChain, HypotheticalDocumentEmbedder
from langchain.prompts import PromptTemplate

from langchain.document_loaders import TextLoader

In [5]:
llm = OpenAI()

embeddings = HypotheticalDocumentEmbedder.from_llm(llm,
                                                   OpenAIEmbeddings(),
                                                   prompt_key = "web_search"
                                                   )

In [6]:
embeddings.llm_chain.prompt

PromptTemplate(input_variables=['QUESTION'], template='Please write a passage to answer the question \nQuestion: {QUESTION}\nPassage:')

In [8]:
import langchain
langchain.debug = True

In [9]:
result = embeddings.embed_query("What items does McDonalds make?")

[llm/start] [1:llm:OpenAI] Entering LLM run with input:
{
  "prompts": [
    "Please write a passage to answer the question \nQuestion: What items does McDonalds make?\nPassage:"
  ]
}
[llm/end] [1:llm:OpenAI] [2.60s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": " McDonald's is a fast-food chain known for its popular menu items that cater to a wide range of tastes and preferences. The restaurant offers a variety of burgers, including the iconic Big Mac, Quarter Pounder with Cheese, and the classic Cheeseburger. In addition to burgers, McDonald's also makes a variety of chicken sandwiches, such as the McChicken and the Buttermilk Crispy Chicken Sandwich. For those looking for a lighter option, McDonald's offers salads, including the Southwest Grilled Chicken Salad and the Caesar Salad. The restaurant also makes a variety of breakfast items, ranging from the classic Egg McMuffin to the indulgent Hotcakes and Sausage. In addition, McDonald's is known for

Multiple Generations

In [10]:
multi_llm = OpenAI(n = 4, best_of =4)

In [11]:
embeddings = HypotheticalDocumentEmbedder.from_llm(
    multi_llm, OpenAIEmbeddings(), "web_search"
)

In [12]:
result = embeddings.embed_query("What is the McDonalds best selling item?")

[llm/start] [1:llm:OpenAI] Entering LLM run with input:
{
  "prompts": [
    "Please write a passage to answer the question \nQuestion: What is the McDonalds best selling item?\nPassage:"
  ]
}
[llm/end] [1:llm:OpenAI] [2.46s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": " The best selling item at McDonald's, the world's largest fast food chain, is the iconic Big Mac. This delicious burger has been a staple on the McDonald's menu since its introduction in 1968 and has remained a fan favorite ever since. The Big Mac consists of two all-beef patties, special sauce, lettuce, cheese, pickles, and onions, all sandwiched between a sesame seed bun. Its popularity can be attributed to its unique combination of flavors and the perfect balance of ingredients. In fact, McDonald's sells an estimated 550 million Big Macs every year, making it the undisputed champion in their menu. However, the fast-food giant also offers a wide variety of other popular items such 

Using own prompts

In [13]:
prompt_template = """Please answer the user's question as a single food
Question: {question}
Answer:

"""

prompt = PromptTemplate(input_variables = ['question'], template = prompt_template)

llm_chain = LLMChain(llm = llm, prompt = prompt)

In [14]:
embeddings = HypotheticalDocumentEmbedder(
    llm_chain = llm_chain,
    base_embeddings = OpenAIEmbeddings()
)

In [15]:
result = embeddings.embed_query(
    "What is the best selling McDonalds item?"
)

[llm/start] [1:llm:OpenAI] Entering LLM run with input:
{
  "prompts": [
    "Please answer the user's question as a single food\nQuestion: What is the best selling McDonalds item?\nAnswer:"
  ]
}
[llm/end] [1:llm:OpenAI] s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "The Big Mac",
        "generation_info": {
          "finish_reason": "stop",
          "logprobs": null
        },
        "type": "Generation"
      }
    ]
  ],
  "llm_output": {
    "token_usage": {
      "total_tokens": 28,
      "prompt_tokens": 25,
      "completion_tokens": 3
    },
    "model_name": "gpt-3.5-turbo-instruct"
  },
  "run": null
}


In [16]:
result

[-0.010497104808417456,
 -0.017987725400382747,
 -0.003831667177604945,
 -0.00104187447277016,
 -0.0008419752884507153,
 0.0016487682385048254,
 -0.031113904351750054,
 -0.008239043484509274,
 -0.0162222172796883,
 -0.02385996354837813,
 0.025407983643861157,
 0.01938222256468059,
 0.02215842238435498,
 -0.013906584390924407,
 -0.01926708129761423,
 0.01921590698721917,
 0.040453193647108095,
 -0.001070659905952066,
 0.020124248202763925,
 -0.00283856816122242,
 -0.005312520292202556,
 -0.0030672527787237707,
 -0.0011418240067222242,
 -0.019446189521351888,
 0.026815273454435233,
 0.004327417145403942,
 0.006678231208411278,
 -0.014955655891378112,
 0.01118795613610573,
 -0.009588762661550171,
 0.02584296341957412,
 -0.013017433076116535,
 -0.028094627489021654,
 0.0034158768369675985,
 -0.01991955096118368,
 0.013522778925606506,
 -0.009601556239148937,
 0.004410574934134653,
 0.02819697610981178,
 -0.004762397852439437,
 0.02206886827248615,
 0.018652991435518552,
 -0.002307635855026

Using HyDE

In [17]:
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma

loaders = [
    TextLoader('/content/blog_posts/blog.langchain.dev_announcing-langsmith_.txt'),
    TextLoader('/content/blog_posts/blog.langchain.dev_benchmarking-question-answering-over-csv-data_.txt'),
    TextLoader('/content/blog_posts/blog.langchain.dev_chat-loaders-finetune-a-chatmodel-in-your-voice_.txt'),
]
docs = []
for l in loaders:
    docs.extend(l.load())

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)

texts = text_splitter.split_documents(docs)

In [18]:
texts

[Document(page_content='URL: https://blog.langchain.dev/announcing-langsmith/\nTitle: Announcing LangSmith, a unified platform for debugging, testing, evaluating, and monitoring your LLM applications\n\nLangChain exists to make it as easy as possible to develop LLM-powered applications.\n\nWe started with an open-source Python package when the main blocker for building LLM-powered applications was getting a simple prototype working. We remember seeing Nat Friedman tweet in late 2022 that there was “not enough tinkering happening.” The LangChain open-source packages are aimed at addressing this and we see lots of tinkering happening now (Nat agrees)–people are building everything from chatbots over internal company documents to an AI dungeon master for a Dungeons and Dragons game.', metadata={'source': '/content/blog_posts/blog.langchain.dev_announcing-langsmith_.txt'}),
 Document(page_content='The blocker has now changed. While it’s easy to build a prototype of an application in ~5 lin

In [19]:
prompt_template = """Please answer the user's question as related to Large Language Models
Question: {question}
Answer:"""

prompt = PromptTemplate(input_variables=["question"], template=prompt_template)

llm_chain = LLMChain(llm=llm, prompt=prompt)

In [20]:
embeddings = HypotheticalDocumentEmbedder(
    llm_chain = llm_chain,
    base_embeddings = OpenAIEmbeddings()
)

In [21]:
docsearch = Chroma.from_documents(texts, embeddings)

query = "What are chat loaders?"
docs = docsearch.similarity_search(query)

[llm/start] [1:llm:OpenAI] Entering LLM run with input:
{
  "prompts": [
    "Please answer the user's question as related to Large Language Models\nQuestion: What are chat loaders?\nAnswer:"
  ]
}
[llm/end] [1:llm:OpenAI] [1.07s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": " Chat loaders are software programs that use large language models to generate responses in conversational chat interfaces, such as messaging apps or virtual assistants. These models are trained on vast amounts of text data and use natural language processing techniques to understand and generate human-like responses. They are becoming increasingly popular in chatbot development, as they can provide more engaging and realistic conversations with users. However, there are concerns about their potential to perpetuate bias and misinformation, as well as the ethical implications of using such powerful language models. ",
        "generation_info": {
          "finish_reason": "stop",